# MedCAT usage
This notebooks shows how to use both MedCAT for entity linking, and MetaCAT for negation detection, on a number of example texts.

In [1]:
from pathlib import Path
from medcat.cat import CAT
from medcat.vocab import Vocab
from medcat.cdb import CDB
from medcat.config import Config
from medcat.meta_cat import MetaCAT

/Users/stan3/miniconda3/envs/negation-detection/lib/python3.8/site-packages/medcat/cat.py:16: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# Input
data_dir = Path.cwd().parents[0] / 'data'
cdb_file = data_dir / 'cdb.dat'
vocab_file = data_dir / 'vocab.dat'
medcat_config_file = Path('medcat_config.txt')
negation_metacat_dir = Path.cwd().parents[0] / 'models' / 'bilstm'

## Setup MedCAT and MetaCAT pipeline

In [3]:
vocab = Vocab.load(vocab_file)
cdb = CDB.load(cdb_file)
config = Config()
config.parse_config_file(medcat_config_file)
negation_metacat = MetaCAT.load(negation_metacat_dir)

# Create MedCAT pipeline
cat = CAT(cdb=cdb, vocab=vocab, config=config, meta_cats=[negation_metacat])

In [4]:
# Test on DL1114 from DCC with negation
text = 'Echo- en rontgenonderzoek van de heup toont geen evidente heupdysplasie.'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

Entity: heup
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.9998283386230469, 'name': 'Negation'}}


Entity: heupdysplasie
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9999861717224121, 'name': 'Negation'}}




In [5]:
# Test on DL1114 from DCC without negation
text = 'Echo- en rontgenonderzoek van de heup toont evidente heupdysplasie.'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

Entity: heup
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.999969482421875, 'name': 'Negation'}}


Entity: heupdysplasie
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.9976205825805664, 'name': 'Negation'}}




## Additional testing

In [6]:
# Test on DL1112 from DCC. When writing this test, the concept "trauma" was not identified.
text = 'Patient kan zich geen trauma herinneren.'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

In [7]:
# Test on NTvG article
# https://www.ntvg.nl/artikelen/acute-buik-op-basis-van-een-wandelende-milt
text = '1 maand na de operatie had patiënte geen buikpijn meer en was zij goed hersteld.'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

Entity: operatie
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.9999604225158691, 'name': 'Negation'}}


Entity: patiënte
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.9994029402732849, 'name': 'Negation'}}


Entity: buikpijn
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9999113082885742, 'name': 'Negation'}}




In [8]:
# Test on NTvG article
# https://www.ntvg.nl/artikelen/een-bezoar-bij-een-vrouw-met-clomipramine-intoxicatie
text = 'Er waren geen tekenen van darmobstructie te zien.'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

Entity: tekenen
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9999995231628418, 'name': 'Negation'}}


Entity: darmobstructie
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9996545910835266, 'name': 'Negation'}}


Entity: zien
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.7851788401603699, 'name': 'Negation'}}




In [9]:
# Test on NTvG article
# https://www.ntvg.nl/artikelen/nieuws/vaker-ziek-na-acute-fase-covid-19
text = 'Alle patiënten werden gematcht met een controlegroep bij wie geen SARS-CoV-2-infectie was geregistreerd.'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

Entity: patiënten
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.9999088644981384, 'name': 'Negation'}}


Entity: controlegroep
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.9999208450317383, 'name': 'Negation'}}


Entity: SARS-CoV-2-infectie
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9999875426292419, 'name': 'Negation'}}




In [10]:
text = 'Er zijn geen bijwerkingen gemeld van de scan'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

Entity: geen bijwerkingen
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9908844232559204, 'name': 'Negation'}}


Entity: scan
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.6775779128074646, 'name': 'Negation'}}




### Check difference space at start
If this causes issues, it can be fixed using `add_prefix_space` during tokenization

In [11]:
text = " Zwelling treedt niet op."
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

Entity: Zwelling
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9410558342933655, 'name': 'Negation'}}




In [12]:
text = "Zwelling treedt niet op."
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")


Entity: Zwelling
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.996150553226471, 'name': 'Negation'}}




### Some more examples

In [13]:
text = "Geen zwelling treedt op."
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

Entity: zwelling
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.999998152256012, 'name': 'Negation'}}




In [14]:
text = "Geen zwelling treedt niet op."
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

Entity: zwelling
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9999976754188538, 'name': 'Negation'}}


